## Problem 1: Galaxy Photometric Redshifts Continued (6 points)

In this exercise, we will return to the galaxy redshifts from problem set 4, equipped with our new tools: gradient boosting and neural networks. Here is the basic information on the data.

The file `Problem_Set_4_Redshifts.csv` contains the following data.
* `<BAND>_FLUX`: Fluxes in different bands. This is what will be used for training. Note that the data has been cleaned, and galaxies with missing fluxes have been removed.
* `Z_PHOT`: Ignore this column.
* `Z_SPEC`: These are spectroscopic redshifts $z_\mathrm{spec}$ measured from the Dark Energy Spectroscopic Instrument (DESI).

(a) Use the `sklearn.ensemble.GradientBoostingRegressor` (default parameters) and compare the feature importance to that of `sklearn.ensemble.RandomForestRegressor` (default parameters). What are the top five most important features? Are the rankings similar?

(b) Perform a $k$-fold cross-validation with $k=10$ to optimize the hyperparameters of `sklearn.ensemble.HistGradientBoostingRegressor` based on the $R^2$ score. Include (+/- 1 standard deviation) error bars derived from the scatter between different folds. For reference, the best-performing gradient boosting regressor performs at least as well as the the best random forest, i.e., $R^2 \gtrsim 0.85$. Here are a few parameters you should explore.
* `learning_rate`
* `max_iter`
* `early_stopping`
* `max_leaf_nodes`

(Tip: Use `n_jobs=-1` in `sklearn.model_selection.GridSearchCV` to parallelize cross-validation.)

(c) Repeat exercise (b) for `sklearn.neural_network.MLPRegressor`. Again, you should aim for an $R^2$ validation score of $R^2 \gtrsim 0.85$. Here are a few parameters you might vary.
* `early_stopping`
* `hidden_layer_sizes`
* `alpha`
* `activation`

I suggest you start with $3$ hidden layers of $100$ neurons each (`hidden_layer_sizes=(100, 100, 100)`). (Hint: Don't forget to scale the input features, for example, via `sklearn.preprocessing.StandardScaler`.)

Part A

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt

df = pd.read_csv("Problem_Set_4_Redshifts.csv")
y = df["Z_SPEC"]
X = df.iloc[:, :35] # all fluxes
y_est = df["Z_PHOT"]

forest = RandomForestRegressor().fit(X, y)
gradient = GradientBoostingRegressor().fit(X, y)

# cleaning up string names
headers = []
for header in X.columns:
    new_header = header.replace("_FLUX", "")
    headers.append(new_header)

# forest plot
fig, ax = plt.subplots(figsize = (9, 5))
ax.tick_params(rotation=90)
plt.bar(headers, forest.feature_importances_)
plt.xlabel('Photometric Bands')
plt.ylabel('Relative Importance')
plt.title('Relative Importance of Photometric Bands to Predicting Spectroscopic Redshifts with Random Forests')
plt.show()

fig, ax = plt.subplots(figsize = (9, 5))
ax.tick_params(rotation=90)
plt.bar(headers, gradient.feature_importances_)
plt.xlabel('Photometric Bands')
plt.ylabel('Relative Importance')
plt.title('Relative Importance of Photometric Bands to Predicting Spectroscopic Redshifts with Gradient Boosting')
plt.show()

[COMPARISON]

Part B

In [ ]:
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np

df = pd.read_csv("Problem_Set_4_Redshifts.csv")
y = df["Z_SPEC"]
X = df.iloc[:, :35] # all fluxes
y_est = df["Z_PHOT"]

k_num = 10
kfold = model_selection.KFold(n_splits=k_num, shuffle=True)

param_grid = dict(early_stopping = [True, False],
                  learning_rate = np.linspace(0.3, 0.5, num=5),
                  max_iter = [5, 7, 9],
                  max_leaf_nodes = [30, 40, 50])
model_search = model_selection.GridSearchCV(HistGradientBoostingRegressor(), param_grid=param_grid, cv=kfold, scoring = "r2", n_jobs=-1)
model_search.fit(X, y)

print(f"Best Params: {model_search.best_params_}\nBest R2 Score: {model_search.best_score_}")

df = pd.DataFrame(model_search.cv_results_)
results = df.rename(columns={f'param_{param}' : param for param in param_grid.keys()})
results = results[[param for param in param_grid.keys()] + ['mean_test_score', 'rank_test_score', 'std_test_score']]
results = results.sort_values(by='rank_test_score')
display(results)


Part C

In [13]:
import pandas as pd
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("Problem_Set_4_Redshifts.csv")
y = df["Z_SPEC"].to_numpy()
X = df.iloc[:, :35] # all fluxes
y_est = df["Z_PHOT"]

X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y.reshape(-1, 1)).flatten()

k_num = 10
kfold = model_selection.KFold(n_splits=k_num, shuffle=True)

param_grid = dict(early_stopping = [True, False],
                  alpha = np.linspace(0.01, 0.1, num=5),
                  hidden_layer_sizes = [[100, 100, 100]],
                  activation = ['relu', 'tanh'],)
model_search = model_selection.GridSearchCV(MLPRegressor(), param_grid=param_grid, cv=kfold, scoring = "r2", n_jobs=-1)
model_search.fit(X, y)

print(f"Best Params: {model_search.best_params_}\nBest R2 Score: {model_search.best_score_}")

df = pd.DataFrame(model_search.cv_results_)
results = df.rename(columns={f'param_{param}' : param for param in param_grid.keys()})
results = results[[param for param in param_grid.keys()] + ['mean_test_score', 'rank_test_score', 'std_test_score']]
results = results.sort_values(by='rank_test_score')
display(results)

/Users/quinnhelfand/Documents/GitHub/School/ISCI-396/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,early_stopping,alpha,hidden_layer_sizes,activation,mean_test_score,rank_test_score,std_test_score
10,True,0.0100,"[100, 100, 100]",tanh,0.890527,1,0.025206
11,False,0.0100,"[100, 100, 100]",tanh,0.890316,2,0.024475
1,False,0.0100,"[100, 100, 100]",relu,0.889216,3,0.037801
13,False,0.0325,"[100, 100, 100]",tanh,0.888240,4,0.028302
12,True,0.0325,"[100, 100, 100]",tanh,0.887625,5,0.030828
14,True,0.0550,"[100, 100, 100]",tanh,0.882007,6,0.028843
15,False,0.0550,"[100, 100, 100]",tanh,0.879919,7,0.028315
16,True,0.0775,"[100, 100, 100]",tanh,0.879135,8,0.032301
17,False,0.0775,"[100, 100, 100]",tanh,0.878276,9,0.028893
19,False,0.1000,"[100, 100, 100]",tanh,0.878184,10,0.032488
